In [1]:
%run ../00_default_options.ipynb

In [2]:
from robotehr.api.training import get_training_results
from robotehr.pipelines import training
from robotehr.models.cohort import Cohort, OnsetDataFrame
from robotehr.models.data import FeaturePipeline
from robotehr.pipelines.supporters.preprocessing import DataLoader

[2020-05-19 16:56:48,340] INFO in __init__: RobotEHR API


In [3]:
cohort = Cohort.load(id=1)

In [4]:
onset_dataframe = OnsetDataFrame.load(id=18)

In [5]:
feature_pipeline = FeaturePipeline.load(id=21)

In [6]:
import morpher.config
from morpher.jobs import *

Using TensorFlow backend.


In [7]:
regexes = [
    'any__Diagnosis__',
    'any__Procedure__', 
    'any__Drug__', 
    'any__Material__', 
    'any__Encounter__', 
    'any__AlcoholUse__', 
    'any__DrugUse__', 
    'any__TobaccoUse__',
    'min__height__', 
    'min__weight__', 
    'min__vitalsign__', 
    'min__measurement__', 
    'min__labvalue__', 
    'max__height__', 
    'max__weight__', 
    'max__vitalsign__', 
    'max__measurement__', 
    'max__labvalue__'
]

agg_func_regex = ""
for r in regexes:
    agg_func_regex += r + "|"
agg_func_regex = agg_func_regex[:-1]

def prepare_data_function(data, target):
    import morpher
    from morpher.jobs import Impute, Scale
    from sklearn.preprocessing import OneHotEncoder
    from enum import Enum
    
    # remove unused features
    del data['medical_record_number']
    del data['mother_account_number']
    del data['date_of_birth']
    del data['month_of_birth']
    del data['patient_ethnic_group']
    del data['religion']
    del data['address_zip']
    del data['deceased_indicator']
    del data['marital_status_code']
    
    class RaceType(str, Enum):
        AFRICAN = 'African'
        AMERICAN_BLACK = 'Black or African-American'
        AMERICAN_NATIVE = 'Native American'
        ASIAN = 'Asian'
        ASIAN_PACIFIC = 'Asian Pacific'
        ASIAN_INDIAN = 'Asian Indian'
        ASIAN_CHINESE = 'Asian Chinese'
        HISPANIC = 'Hispanic or Latino'
        OTHER = 'Other'
        WHITE = 'White'
    
    RACE_MAPPING = {
        RaceType.AFRICAN: [
            'Cape Verdian',
            'Congolese',
            'Eritrean',
            'Ethiopian',
            'Gabonian',
            'Ghanaian',
            'Guinean',
            'Ivory Coastian',
            'Kenyan',
            'Liberian',
            'Madagascar',
            'Malian',
            'Nigerian',
            'Other: East African',
            'Other: North African',
            'Other: South African',
            'Other: West African',
            'Senegalese',
            'Sierra Leonean',
            'Somalian',
            'Sudanese',
            'Tanzanian',
            'Togolese',
            'Ugandan',
            'Zimbabwean'
        ],
        RaceType.AMERICAN_BLACK: [
            'African American (Black)',
            'African-American',
            'Black Or African-American',
            'Black or African - American',
        ],
        RaceType.AMERICAN_NATIVE: [
            'American (Indian/Alaskan)',
            'Native American'
        ],
        RaceType.ASIAN: [
            'Asian',
            'Bangladeshi',
            'Bhutanese',
            'Burmese',
            'Cambodian',
            'Hmong',
            'Indonesian',
            'Japanese',
            'Korean',
            'Laotian',
            'Malaysian',
            'Maldivian',
            'Nepalese',
            'Okinawan',
            'Pakistani',
            'Singaporean',
            'Taiwanese',
            'Thai',
            'Vietnamese',
            'Yapese'
        ],
        RaceType.ASIAN_PACIFIC: [
            'Asian (Pacific Islander)',
            'Carolinian',
            'Chamorro',
            'Chuukese',
            'Fijian',
            'Filipino',
            'Guamanian',
            'Guamanian Or Chamorro',
            'Guamanian or Chamorro',
            'Iwo Jiman',
            'Kiribati',
            'Kosraean',
            'Mariana Islander',
            'Marshallese',
            'Melanesian',
            'Micronesian',
            'Native Hawaiian',
            'New Hebrides',
            'Other Pacific Islander',
            'Pacific Islander',
            'Palauan',
            'Pohnpeian',
            'Polynesian',
            'Saipanese',
            'Samoan',
            'Papua New Guinean',
            'Tahitian',
            'Tokelauan',
            'Tongan'
        ],
        RaceType.ASIAN_INDIAN: [
            'Asian Indian',
            'Sri Lankan',
            'Sri lankan',
            'West Indian'
        ],
        RaceType.ASIAN_CHINESE: [
            'Chinese',
        ],
        RaceType.HISPANIC: [
            'Barbadian',
            'Dominica Islander',
            'Grenadian',
            'Haitian',
            'Hispanic/Latino',
            'Jamaican',
            'St Vincentian',
            'Trinidadian'
        ],
        RaceType.OTHER: [
            '',
            'Aa',
            'Ab',
            'Af',
            'Ag',
            'Ak',
            'Al',
            'Ap',
            'Ar',
            'Av',
            'Ay',
            'B',
            'B1',
            'B2',
            'B3',
            'B4',
            'B5',
            'B6',
            'B7',
            'B8',
            'B9',
            'Ba',
            'Bb',
            'Bc',
            'Bd',
            'Be',
            'Bf',
            'Bg',
            'Bh',
            'Bj',
            'Bk',
            'Bm',
            'Bn',
            'Bo',
            'Bp',
            'Bq',
            'Br',
            'Bs',
            'Bt',
            'Bu',
            'Bv',
            'Bw',
            'Bx',
            'By',
            'Bz',
            'I',
            'MSDW_NOT APPLICABLE',
            'MSDW_OTHER',
            'MSDW_UNKNOWN',
            'NOT AVAILABLE',
            'Non Hispanic',
            'O',
            'Other',
            'Pk',
            'Pl',
            'Pm',
            'Po',
            'Ps',
            'Pv',
            'U',
            'Unk',
            'Unknown',
            'W'
        ],
        RaceType.WHITE: [
            'Caucasian (White)',
            'White'
        ]
    }
    
    # value mapping
    data['race'] = (
        data.race.map({
            label: cat for cat, labels in RACE_MAPPING.items()
            for label in labels
        }).astype(pd.api.types.CategoricalDtype(RaceType))
    )
    
    # pre-encoding nan handling
    for column in ['gender', 'race']:
        enc = OneHotEncoder(sparse=False)
        transformed_data = enc.fit_transform(data[[column]])
        transformed_columns = pd.DataFrame(
            data=transformed_data, 
            columns=[f'{column}_{c}' for c in enc.categories_[0]]
        )
        data = data.join(transformed_columns)
        del data[column]
        
    label_encoded_data = data.copy()
    
    label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]] = label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]].fillna(False)
    
    scaled_data, _ = Scale().execute(data=label_encoded_data, target=target)
    
    imputed_data, _ = Impute().execute(data=scaled_data, imputation_method=morpher.config.imputers.DEFAULT)
    
    return imputed_data

data_loader = DataLoader(agg_func_regex, prepare_data_function)

In [8]:
from itertools import product

In [9]:
thresholds_numeric = [(x / 100) for x in range(5, 100, 5)]
observation_windows_numeric = [[x, -1] for x in range(-361, -1, 30)]
thresholds_occurring = [(x / 100) for x in range(5, 100, 5)]
observation_windows_occurring = [[x, -1] for x in range(-361, -1, 30)]

In [10]:
iterator = product(
    thresholds_numeric,
    observation_windows_numeric,
    thresholds_occurring,
    observation_windows_occurring
)

configs = [x for x in iterator if x[0] == x[2] and x[1] == x[3]]
targets = ['other_viral_infection_onset_from_0_days_after_to_365_days_after']
algorithms = [morpher.config.algorithms.GBDT, morpher.config.algorithms.RF, morpher.config.algorithms.DT, morpher.config.algorithms.LR]
samplers = [morpher.config.samplers.RANDOM, morpher.config.samplers.URANDOM, morpher.config.samplers.BORDERLINE, morpher.config.samplers.SMOTE, morpher.config.samplers.NOSAMPLER]

In [13]:
pipeline = training.execute(
    comment='baseline-4 ovi tw binning - all features #2',
    version='34.1.1',
    cohort=cohort,
    onset_dataframe=onset_dataframe,
    feature_pipeline=feature_pipeline,
    data_loader=data_loader,
    observation_iterator=configs,
    targets=targets,
    algorithms=algorithms,
    samplers=samplers,
    feature_type_occurring="occurring",
    feature_type_numeric="numeric_binned",
    bin_size=30,
    rfe__run=True,
    rfe__step_size=50
)

Fetching data for <fiber.condition.mrns.MRNs object at 0x7f4d0d23c898>
Fetching data for Patient (...)


Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
Data with input dtype bool, float64 were all converted to float64 by StandardScaler.


Fitting estimator with 3102 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 667.967ms

Fitting estimator with 3052 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 568.8ms

Fitting estimator with 3002 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 774.634ms

Fitting estimator with 2952 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 891.786ms

Fitting estimator with 2902 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 834.86ms

Fitting estimator with 2852 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 104.654ms

Fitting estimator with 2802 features.
*** Training of model '

*** Training of classifier ready. Time elapsed: 850.025ms

Fitting estimator with 2652 features.
*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 926.749ms

Fitting estimator with 2602 features.
*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 982.39ms

Fitting estimator with 2552 features.
*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 938.141ms

Fitting estimator with 2502 features.
*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 17.215ms

Fitting estimator with 2452 features.
*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 895.58ms

Fitting estimator with 2402 features.
*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 888.972ms

Fitting estimator with 235

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 306.427ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 383.407ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 274.547ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 254.793ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 141.317ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 145.693ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 106.936ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 282.901ms

*** Training of model 'GradientBoostingClassifie

...

Columns (2,3,12,13,19,20,23,28,31,32,33,34,35,39,43,47,52,55,56,62,63,64,65,69,70,71,72,79,80,82,83,84,86,87,88,89,91,97,98,99,102,103,104,105,107,108,109,110,111,113,114,115,116,117,118,120,127,143,144,148,150,155,156,158,164,167,168,170,173,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,204,205,206,213,214,215,216,218,221,222,224,225,226,227,229,230,231,232,233,234,235,236,237,238,240,241,242,244,245,246,247,248,249,250,251,252,253,257,258,260,261,262,263,264,266,267,268,273,274,282,283,285,286,287,289,291,292,293,296,311,314,318,322,324,325,326,341,342,343,345,347,349,350,351,353) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7f4d0cb15d68>
Fetching data for Patient (...)


Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
Data with input dtype bool, float64 were all converted to float64 by StandardScaler.


*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 276.726ms

*** Training of model 'GradientBoostingClassifier' started.


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
pipeline.id

198

# Results

In [19]:
get_training_results(198, sort_by='auc_roc_mean', metrics=['auc_roc_mean', 'auc_roc_std', 'precision_mean', 'recall_mean', 'auc_prc_mean'], n_rows=10000, response_type="pandas")

,algorithm,auc_prc_mean,auc_roc_mean,auc_roc_std,precision_mean,recall_mean,sampler,target,threshold_numeric,threshold_occurring,window_start_numeric,window_start_occurring
0,LogisticRegression,0.694585,0.808255,0.029705,0.583653,0.637548,NoSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-361,-361
1,LogisticRegression,0.691757,0.804541,0.030117,0.585684,0.641026,BorderlineSMOTE,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-361,-361
2,LogisticRegression,0.690998,0.804394,0.030765,0.587617,0.623814,RandomOverSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-361,-361
3,LogisticRegression,0.691083,0.803555,0.028825,0.581294,0.642706,SMOTE,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-361,-361
4,LogisticRegression,0.657182,0.792901,0.039541,0.572025,0.634055,RandomOverSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-211,-211
5,LogisticRegression,0.657987,0.790956,0.035046,0.565791,0.628912,NoSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-211,-211
6,GradientBoostingDecisionTree,0.676073,0.790688,0.038754,0.768718,0.407162,NoSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-331,-331
7,LogisticRegression,0.646713,0.786717,0.036511,0.547126,0.644312,SMOTE,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-211,-211
8,GradientBoostingDecisionTree,0.680104,0.786397,0.036676,0.611121,0.542912,RandomOverSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-331,-331
9,GradientBoostingDecisionTree,0.673237,0.785645,0.032826,0.652555,0.482832,SMOTE,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-331,-331
